In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import os
from imblearn.over_sampling import SMOTE
import glob

In [77]:
# Load in your data here
folder_path = 'Minor_Disorders'
file_list = glob.glob(os.path.join(folder_path, '*.csv'))

In [78]:
folder_path = 'Minor_Disorders'
file_list = glob.glob(os.path.join(folder_path, '*.csv'))
for file in file_list:
    basename = os.path.basename(file)  # e.g. 'Acute_subjects.csv'
    disorder_name = basename.split('_')[0]
    print(f'Processing file: {file} as disorder: {disorder_name}')  
    data = pd.read_csv(file)
    bands = [
        ('Delta', data.iloc[:, list(range(0,22))+list(range(118,289))+[-1]]),
        ('Theta', data.iloc[:, list(range(0,3))+list(range(22,41))+list(range(289,460))+[-1]]),
        ('Alpha', data.iloc[:, list(range(0,3)) + list(range(41,60))+list(range(460,631))+[-1]]),
        ('Beta', data.iloc[:, list(range(0,3)) + list(range(60,79))+list(range(631,802))+[-1]]),
        ('HighBeta', data.iloc[:, list(range(0,3)) + list(range(79,98))+list(range(802,973))+[-1]]),
        ('Gamma', data.iloc[:, list(range(0,3)) + list(range(98,117))+list(range(973,1144))+[-1]]),
        ('All', data.iloc[:, list(range(0,117))+list(range(118,1145))])
    ]
    all_model_data = []

#bands = [delta, theta, alpha, beta, highbeta, gamma, allb]
    for label, band in bands:
        # Data preprocessing
        print(f"\n{label} columns:\n", band.columns.tolist())
        X = band.drop('Class', axis=1)
        y = band['Class']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, stratify=None)
        #using smote to oversample
        #smote = SMOTE(random_state=0)
        #X_train, y_train = smote.fit_resample(X, y)
        #y_train.hist()
        pca = PCA(0.95)
        X = X.dropna()
        #X_pca = pca.fit_transform(X_train)
        X_pca.shape
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)
        #X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.1, random_state=30)
        #using smote to oversample
        #smote = SMOTE(random_state=0)
        #X_train_pca, y_train = smote.fit_resample(X_pca, y)
        #y_train.hist()
        svclassifier = SVC(kernel='linear',probability=True)
        svclassifier.fit(X_train, y_train)
        y_pred = svclassifier.predict(X_test)
        print('SVM accuracy:', svclassifier.score(X_test, y_test))
        print('SVM classification report:\n', classification_report(y_test, y_pred))
        y_scores = svclassifier.predict_proba(X_test)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_test, y_scores)
        # Calculate AUC
        svm_auc = auc(fpr, tpr)
        print('SVM AUC value:', svm_auc)
        newsvm=SVC(kernel='linear', probability=True)
        newsvm.fit(X_train_pca, y_train)
        y_pred = newsvm.predict(X_test_pca)
        print('\nSVM accuracy for PCA:', newsvm.score(X_test_pca, y_test))
        print('SVM classification report for PCA:\n', classification_report(y_test, y_pred))
        y_scores = newsvm.predict_proba(X_test_pca)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_test, y_scores)
        pcasvm_auc = auc(fpr, tpr)
        print('SVM AUC value for PCA:', pcasvm_auc)
        
        knn = KNeighborsClassifier(n_neighbors=10)
        knn.fit(X_train, y_train)
        print('KNN accuracy:', knn.score(X_test, y_test))
        y_pred = knn.predict(X_test)
        print('KNN classification report:\n',classification_report(y_test, y_pred))
        y_scores = knn.predict_proba(X_test)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_test, y_scores)
        knn_auc = auc(fpr, tpr)
        print('KNN AUC value:', knn_auc)
        pknn = KNeighborsClassifier(n_neighbors=10)
        pknn.fit(X_train_pca, y_train)
        print('\nKNN accuracy for PCA:', pknn.score(X_test_pca, y_test))
        y_pred = pknn.predict(X_test_pca)
        print('KNN classification report for PCA\n', classification_report(y_test, y_pred))
        y_scores = pknn.predict_proba(X_test_pca)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_test, y_scores)
        pcaknn_auc = auc(fpr, tpr)
        print('KNN AUC value for PCA',pcaknn_auc)
        
        model = RandomForestClassifier(n_estimators=40)
        model.fit(X_train, y_train)
        print('Random Forest accuracy:', model.score(X_test, y_test))
        y_pred = model.predict(X_test)
        print('Random Forest classification report\n',classification_report(y_test, y_pred))
        y_scores = model.predict_proba(X_test)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_test, y_scores)
        rf_auc = auc(fpr, tpr)
        print('Random Forest AUC value:', rf_auc)
        pmodel = RandomForestClassifier(n_estimators=40)
        pmodel.fit(X_train_pca, y_train)
        print('\nRandom Forest accuracy for PCA:', pmodel.score(X_test_pca, y_test))
        y_pred = pmodel.predict(X_test_pca)
        print('Random Forest classification report for PCA:\n',classification_report(y_test, y_pred))
        y_scores = pmodel.predict_proba(X_test_pca)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_test, y_scores)
        pcarf_auc = auc(fpr, tpr)
        print('Random Forest AUC value for PCA:', pcarf_auc)
        
        lg = LogisticRegression()
        lg.fit(X_train, y_train)
        print('Logistic Regression accuracy:', lg.score(X_test, y_test))
        y_pred = lg.predict(X_test)
        print('Logistic Regression classification report:\n',classification_report(y_test, y_pred))
        y_scores = lg.predict_proba(X_test)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_test, y_scores)
        lg_auc = auc(fpr, tpr)
        print('Logistic Regression AUC value:',lg_auc)
        plg = LogisticRegression()
        plg.fit(X_train_pca, y_train)
        print('\nLogistic Regression accuracy for PCA:',plg.score(X_test_pca, y_test))
        y_pred = plg.predict(X_test_pca)
        print('Logistic Regression classification report for PCA:\n',classification_report(y_test, y_pred))
        y_scores = plg.predict_proba(X_test_pca)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_test, y_scores)
        pcalg_auc = auc(fpr, tpr)
        print('Logistic Regression AUC value for PCA',pcalg_auc)
        
        clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
        clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])
        y_pred = clf.predict(X_test)
        print('XGB accuracy:',clf.score(X_test, y_test))
        print('XGB Classification report:\n',classification_report(y_test, y_pred))
        y_scores = clf.predict_proba(X_test)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_test, y_scores)
        xgb_auc = auc(fpr, tpr)
        print('XGB AUC value:',xgb_auc)
        pxgb = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
        pxgb.fit(X_train_pca, y_train, eval_set=[(X_test_pca, y_test)])
        print('\nXGB accuracy for PCA:', pxgb.score(X_test_pca, y_test))
        y_pred = pxgb.predict(X_test_pca)
        print('XGB classification report for PCA:\n',classification_report(y_test, y_pred))
        y_scores = pxgb.predict_proba(X_test_pca)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_test, y_scores)
        pxgb_auc = auc(fpr, tpr)
        print('XBG AUC for PCA:', pxgb_auc)
        cm = confusion_matrix(y_test, y_pred)
        print(cm)
        model_row = [
            f"{disorder_name}_{label}_PSDFC",
            svclassifier.score(X_test, y_test), svm_auc,
            knn.score(X_test, y_test), knn_auc,
            model.score(X_test, y_test), rf_auc,
            lg.score(X_test, y_test), lg_auc,
            clf.score(X_test, y_test), xgb_auc
        ]
        pca_row = [
            f"{disorder_name}_{label}_PSDFCPCA",
            newsvm.score(X_test_pca, y_test), pcasvm_auc,
            pknn.score(X_test_pca, y_test),pcaknn_auc,
            pmodel.score(X_test_pca, y_test),pcarf_auc,
            plg.score(X_test_pca, y_test),pcalg_auc,
            pxgb.score(X_test_pca, y_test), pxgb_auc
        ]
        all_model_data.append(model_row)
        all_model_data.append(pca_row)
        for row in all_model_data:
            print(row, type(row), len(row) if hasattr(row, '__len__') else 'Not iterable')
columns = ['Combination', 'SVM Accuracy', 'SVM AUC', 
           'KNN Accuracy', 'KNN AUC', 
           'Random Forest Accuracy','Random Forest AUC', 
           'Logistic Regression Accuracy', 'Logistic Regression AUC', 
           'XGB Accuracy', 'XGB AUC']
    
df = pd.DataFrame(all_model_data, columns=columns)
df.to_csv('minor_disorder_PSDFCdata.csv', mode='a',header=not os.path.exists('minor_disorder_PSDFCdata.csv'),index=False)

Processing file: Minor_Disorders/Acute.csv as disorder: Acute.csv

Delta columns:
 ['age', 'education', 'IQ', 'AB.A.delta.a.FP1', 'AB.A.delta.b.FP2', 'AB.A.delta.c.F7', 'AB.A.delta.d.F3', 'AB.A.delta.e.Fz', 'AB.A.delta.f.F4', 'AB.A.delta.g.F8', 'AB.A.delta.h.T3', 'AB.A.delta.i.C3', 'AB.A.delta.j.Cz', 'AB.A.delta.k.C4', 'AB.A.delta.l.T4', 'AB.A.delta.m.T5', 'AB.A.delta.n.P3', 'AB.A.delta.o.Pz', 'AB.A.delta.p.P4', 'AB.A.delta.q.T6', 'AB.A.delta.r.O1', 'AB.A.delta.s.O2', 'COH.A.delta.a.FP1.b.FP2', 'COH.A.delta.a.FP1.c.F7', 'COH.A.delta.a.FP1.d.F3', 'COH.A.delta.a.FP1.e.Fz', 'COH.A.delta.a.FP1.f.F4', 'COH.A.delta.a.FP1.g.F8', 'COH.A.delta.a.FP1.h.T3', 'COH.A.delta.a.FP1.i.C3', 'COH.A.delta.a.FP1.j.Cz', 'COH.A.delta.a.FP1.k.C4', 'COH.A.delta.a.FP1.l.T4', 'COH.A.delta.a.FP1.m.T5', 'COH.A.delta.a.FP1.n.P3', 'COH.A.delta.a.FP1.o.Pz', 'COH.A.delta.a.FP1.p.P4', 'COH.A.delta.a.FP1.q.T6', 'COH.A.delta.a.FP1.r.O1', 'COH.A.delta.a.FP1.s.O2', 'COH.A.delta.b.FP2.c.F7', 'COH.A.delta.b.FP2.d.F3', 'COH.A

/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/novo/miniforge3/


SVM accuracy for PCA: 0.6153846153846154
SVM classification report for PCA:
               precision    recall  f1-score   support

           0       1.00      0.62      0.76        13
           1       0.00      0.00      0.00         0

    accuracy                           0.62        13
   macro avg       0.50      0.31      0.38        13
weighted avg       1.00      0.62      0.76        13

SVM AUC value for PCA: nan
KNN accuracy: 1.0
KNN classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        13

    accuracy                           1.00        13
   macro avg       1.00      1.00      1.00        13
weighted avg       1.00      1.00      1.00        13

KNN AUC value: nan

KNN accuracy for PCA: 1.0
KNN classification report for PCA
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        13

    accuracy                           1.00        13
   m

/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/novo/miniforge3/

Random Forest classification report for PCA:
               precision    recall  f1-score   support

           0       1.00      0.85      0.92        13
           1       0.00      0.00      0.00         0

    accuracy                           0.85        13
   macro avg       0.50      0.42      0.46        13
weighted avg       1.00      0.85      0.92        13

Random Forest AUC value for PCA: nan
Logistic Regression accuracy: 0.6153846153846154
Logistic Regression classification report:
               precision    recall  f1-score   support

           0       1.00      0.62      0.76        13
           1       0.00      0.00      0.00         0

    accuracy                           0.62        13
   macro avg       0.50      0.31      0.38        13
weighted avg       1.00      0.62      0.76        13

Logistic Regression AUC value: nan

Logistic Regression accuracy for PCA: 0.6153846153846154
Logistic Regression classification report for PCA:
               precision  

...

[0]	validation_0-logloss:0.58610
[1]	validation_0-logloss:0.51455
[2]	validation_0-logloss:0.50969
[3]	validation_0-logloss:0.46969
[4]	validation_0-logloss:0.43732
[5]	validation_0-logloss:0.45159
XGB accuracy: 0.8421052631578947
XGB Classification report:
               precision    recall  f1-score   support

           0       1.00      0.77      0.87        13
           1       0.67      1.00      0.80         6

    accuracy                           0.84        19
   macro avg       0.83      0.88      0.83        19
weighted avg       0.89      0.84      0.85        19

XGB AUC value: 0.9102564102564102
[0]	validation_0-logloss:0.66458
[1]	validation_0-logloss:0.65645
[2]	validation_0-logloss:0.69805
[3]	validation_0-logloss:0.72585

XGB accuracy for PCA: 0.5263157894736842
XGB classification report for PCA:
               precision    recall  f1-score   support

           0       0.70      0.54      0.61        13
           1       0.33      0.50      0.40         6

    ac

/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


XGB Classification report:
               precision    recall  f1-score   support

           0       0.90      0.82      0.86        11
           1       0.78      0.88      0.82         8

    accuracy                           0.84        19
   macro avg       0.84      0.85      0.84        19
weighted avg       0.85      0.84      0.84        19

XGB AUC value: 0.8693181818181819
[0]	validation_0-logloss:0.58996
[1]	validation_0-logloss:0.59900

XGB accuracy for PCA: 0.7368421052631579
XGB classification report for PCA:
               precision    recall  f1-score   support

           0       0.88      0.64      0.74        11
           1       0.64      0.88      0.74         8

    accuracy                           0.74        19
   macro avg       0.76      0.76      0.74        19
weighted avg       0.77      0.74      0.74        19

XBG AUC for PCA: 0.8863636363636364
[[7 4]
 [1 7]]
['Alcohol.csv_Delta_PSDFC', 0.7368421052631579, 0.6136363636363636, 0.631578947368421, 0.

/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

[0]	validation_0-logloss:0.59190
[1]	validation_0-logloss:0.55679
[2]	validation_0-logloss:0.50065
[3]	validation_0-logloss:0.47879
[4]	validation_0-logloss:0.48043
[5]	validation_0-logloss:0.44264
[6]	validation_0-logloss:0.42757
[7]	validation_0-logloss:0.38946
[8]	validation_0-logloss:0.37618
[9]	validation_0-logloss:0.39333
XGB accuracy: 0.8421052631578947
XGB Classification report:
               precision    recall  f1-score   support

           0       0.75      1.00      0.86         9
           1       1.00      0.70      0.82        10

    accuracy                           0.84        19
   macro avg       0.88      0.85      0.84        19
weighted avg       0.88      0.84      0.84        19

XGB AUC value: 0.9555555555555556
[0]	validation_0-logloss:0.64132
[1]	validation_0-logloss:0.66193
[2]	validation_0-logloss:0.62731
[3]	validation_0-logloss:0.63101

XGB accuracy for PCA: 0.5789473684210527
XGB classification report for PCA:
               precision    recall  f1-